In [7]:
# classifier: cats vs. dogs


Using TensorFlow backend.


In [8]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

11493376/11490434 [==============================] - 2s 0us/step


In [10]:
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

(60000, 28, 28)
(60000,)
(10000, 28, 28)
(10000,)


In [16]:
#Initialziation
IMG_SIZE=28

In [17]:
# ---------------------
# construct the cnn model for this project
# ---------------------
import tflearn
import tensorflow as tf

from tflearn.layers.conv import conv_2d, max_pool_2d
from tflearn.layers.core import input_data, dropout, fully_connected, flatten
from tflearn.layers.estimator import regression
tf.nn.sigmoid_cross_entropy_with_logits


def conv_net():
    convnet = input_data(shape = [None, IMG_SIZE, IMG_SIZE, 1],
                        name = 'input')
    # conv_2d(incoming, nb_filter, filter_size, ..., activation)
    convnet = conv_2d(convnet, 32, 5, activation = 'relu')
    # max_pool_2d(incoming, kernel_size, ...)
    convnet = max_pool_2d(convnet, 5)
    
    convnet = conv_2d(convnet, 64, 5, activation = 'relu')
    convnet = max_pool_2d(convnet, 5)
   
    #flatten the layer
    convnet = tflearn.flatten(convnet)
    # fully_connected(incoming, n_units, activation, ...)
    convnet = fully_connected(convnet, 1024, activation = 'relu')
    # dropout(incoming, keep_prob is drop_prob + keep+prob)
    #convnet = dropout(convnet, 0.8)
    
    #standard recommendation for the Net arch    
    convnet = fully_connected(convnet, 2, activation = 'softmax')
    convnet = dropout(convnet, 0.8)
    
    #logits
    logits = tf.layers.dense(inputs=dropout, units=8)
    
    #regression(incoming, optimizer, learning_rate, loss, name, ...)
    convnet = regression(convnet, optimizer = 'adam', learning_rate = LR,
                             loss = 'categorical_crossentropy', name = 'targets')

    return convnet

In [18]:
# ---------------------
# define the model
# ---------------------
convnet = conv_net()
model = tflearn.DNN(convnet, tensorboard_dir = 'log')


W0626 21:35:21.254591 140735728333696 deprecation_wrapper.py:119] From /usr/local/lib/python3.7/site-packages/tflearn/layers/core.py:81: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0626 21:35:21.258888 140735728333696 deprecation.py:506] From /usr/local/lib/python3.7/site-packages/tflearn/initializations.py:119: calling UniformUnitScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0626 21:35:21.259881 140735728333696 deprecation.py:323] From /usr/local/lib/python3.7/site-packages/tensorflow/python/util/deprecation.py:507: UniformUnitScaling.__init__ (from tensorflow.python.ops.init_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.initializers.variance_scaling instead with distribution=uniform to get equivalent beha

AttributeError: 'function' object has no attribute 'shape'

In [10]:
# ---------------------
# train the model
# ---------------------

# load model if exists
if os.path.exists('{}.meta'.format(MODEL_NAME)):
    model.load(MODEL_NAME)
    print ('loaded existing model', MODEL_NAME)
else:
    # Loading the training data-set after pre-processing
    # and splitting the testing data and validation data
    
    train_data = load_train_data(False)
    print(np.array(train_data).shape)
    divide = (len(train_data)/5)
    for i in range(5):
        start_id = (divide * i)
        print("startid: " + str(start_id))
        end_id = (divide * (i + 1) -1)
        print("endid: " + str(end_id))
        validation = train_data[start_id: end_id]
        if (i == 0):
            train = train_data[end_id +1 : (len(train_data)-1)]
            #print("Train0:" + str(train))
        if (i == 1 and i == 2 and i == 3):
            train = train_data[0: (start_id)-1].merge[end_id +1 : (len(train_data)-1)]
        if(i == 4):
            train = train = train_data[0 : start_id -1]
        # Setting up the features and lables
        # x-Features & y-Labels
        train_x = np.array([i[0] for i in train]).reshape(-1, IMG_SIZE, IMG_SIZE, 1)
        train_y = [i[1] for i in train]
        validation_x = np.array([i[0] for i in validation]).reshape(-1, IMG_SIZE, IMG_SIZE, 1)
        validation_y = [i[1] for i in validation]
        # Training the CNN model with the data-sets
        model.fit({'input': train_x}, {'targets': train_y}, n_epoch = 15,
                            validation_set = ({'input': validation_x}, {'targets': validation_y}),
                            snapshot_step = 500, show_metric = True, run_id = MODEL_NAME)

    # save the trained model
    #model.save(MODEL_NAME)
    #enough training/testing sample of each class
    #JSMA
    #debug on a different MNISTdataset/ standard arch

Training Step: 7733  | total loss: 11.59247 | time: 5.103s
| Adam | epoch: 077 | loss: 11.59247 - acc: 0.4965 -- iter: 3648/6403


KeyboardInterrupt: 

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
'''refined_x=train_x.flatten().reshape(7505, 2500)
train_y_labels=[]
for i in range(len(train_y)):
    train_y_labels.append(train_y[i][0])
    train_y_labels=np.array(train_y_labels)
    logisticRegr = LogisticRegression()
    logisticRegr.fit(refined_x,train_y_labels)'''

In [ ]:
'''prediction=logisticRegr.predict(refined_x)
count=0
total_train = len(prediction)
for i in range(len(prediction)):
    if (prediction[i]==train_y_labels[i]):
        print("prediction{}:, Train{}: ".format(prediction, train_y_labels))
        count=count+1 
accuracy = (1.0 * count / total_train)             
print("The training accuracy is {}.".format(accuracy))'''

In [ ]:
# the model acheives 90% accuracy, save it if not saved yet
if not os.path.exists('{}.meta'.format(MODEL_NAME)):
    model.save(MODEL_NAME)

In [ ]:
# ---------------------
# load testing data
# ---------------------
def load_test_data(load_exist):
    testset = None
    if load_exist and os.path.exists('testset-{}.npy'.format(IMG_SIZE)):
        # if want to load an existing testing set
        testset = np.load('testset-{}.npy'.format(IMG_SIZE), allow_pickle = True)
    else: 
        testset = process_test_data()
    return testset

In [ ]:
test_data = load_test_data(False)
test_x = np.array([i[0] for i in test_data]).reshape(-1, IMG_SIZE, IMG_SIZE, 1)
test_y = [i[1] for i in test_data] 

predictions = model.predict(test_x)
total_test_samples = 0 #len(test_data)

counter = 0
for prediction, true_label in zip(predictions, test_y):
    total_test_samples +=1
    predicted_class = np.argmax(prediction) # [0.6, 0.4] #predicts whichever is max
    actual_class = np.argmax(true_label)
    print("predicted:" + str(prediction))
    print("true_lable:"+str(true_label))# [0, 1], [1, 0]
    if(predicted_class == actual_class):
        counter+=1
        
    print("predicate class: {}, True label: {}, counter {} ".format(predicted_class, actual_class, counter))

# For integers, ‘/’ operator returns the quotient. For example, 3/10 returns 0.

# So, add 1.0 before counter to convert it to a float number. 

accuracy = 1.0 * counter / total_test_samples
print(total_test_samples)

print("The testing accuracy is {}.".format(accuracy))


In [ ]:
# -------------------
# use trained model to predict given image
# -------------------
import matplotlib.pyplot as plt

test_data = load_test_data(True)
fig = plt.figure(figsize = (10, 8))

# test and plot 12 images
for i, data in enumerate(test_data[:12]):
    # cat: [1, 0]; dog: [0, 1]
    img_data = data[0]
    img_idx = data[1]
    
    sub_fig = fig.add_subplot(3, 4, i + 1)
    
    orig_data = img_data
    # flatten image
    data = img_data.reshape(IMG_SIZE, IMG_SIZE, 1)
    
    output = model.predict([data])[0]
    
    if np.argmax(output) == 1:
        pred_label = 'dog'
    else:
        pred_label = 'cat'
    
    sub_fig.imshow(orig_data, cmap = 'gray')
    
    # predicted label and confidence
    plt.title(pred_label + '; ' + str(np.max(output)))
    sub_fig.get_xaxis().set_visible(False)
    sub_fig.get_yaxis().set_visible(False)
    
plt.show()